In [33]:
import pandas as pd
import numpy as np

In [34]:
df = pd.read_csv('data_combined.csv')
df.head()

,Unnamed: 0,date_crawl,city,uid,book_date,price,event,rating,provider,source,event_name,tweets,retweets,likes,replies,amplification_rate,conversation_rate,applause_rate
0,0,2023-03-30,La Palma,title_44546103,2023-05-20,90,True,4.78,privat,airbnb,MLF23,18.0,8.0,68.0,1.0,0.444444,0.055556,3.777778
1,1,2023-03-30,La Palma,title_14832235,2023-05-20,86,True,4.78,privat,airbnb,MLF23,18.0,8.0,68.0,1.0,0.444444,0.055556,3.777778
2,2,2023-03-30,La Palma,title_22183776,2023-05-20,63,True,4.90,privat,airbnb,MLF23,18.0,8.0,68.0,1.0,0.444444,0.055556,3.777778
3,3,2023-03-30,La Palma,title_26944967,2023-05-20,109,True,4.76,privat,airbnb,MLF23,18.0,8.0,68.0,1.0,0.444444,0.055556,3.777778
4,4,2023-03-30,La Palma,title_741718109680848267,2023-05-20,46,True,4.85,privat,airbnb,MLF23,18.0,8.0,68.0,1.0,0.444444,0.055556,3.777778


In [35]:
df.columns

Index(['Unnamed: 0', 'date_crawl', 'city', 'uid', 'book_date', 'price',
       'event', 'rating', 'provider', 'source', 'event_name', 'tweets',
       'retweets', 'likes', 'replies', 'amplification_rate',
       'conversation_rate', 'applause_rate'],
      dtype='object')

In [36]:
df.drop('Unnamed: 0', axis=1, inplace=True)
df.drop('event_name', axis=1, inplace=True)
df.drop('uid', axis=1, inplace=True)



In [37]:
df['book_date'] = df['book_date'].astype('datetime64[ns]')
df['date_crawl'] = df['date_crawl'].astype('datetime64[ns]')
df['year'] = df['book_date'].dt.year
df['month'] = df['book_date'].dt.month
df['day'] = df['book_date'].dt.day
df['year_crawl'] = df['date_crawl'].dt.year
df['month_crawl'] = df['date_crawl'].dt.month
df['day_crawl'] = df['date_crawl'].dt.day
df.drop('book_date', axis=1, inplace=True)
df.drop('date_crawl', axis=1, inplace=True)

In [38]:
#get dummies for categorical variables city
df = pd.get_dummies(df, columns=['city'], drop_first=True)
df = pd.get_dummies(df, columns=['event'], drop_first=True)
df = pd.get_dummies(df, columns=['provider'], drop_first=True)
df = pd.get_dummies(df, columns=['source'], drop_first=True)

In [39]:
df.drop('retweets', axis=1, inplace=True)
df.drop('likes', axis=1, inplace=True)
df.drop('replies', axis=1, inplace=True)
df.drop('amplification_rate', axis=1, inplace=True)
df.drop('conversation_rate', axis=1, inplace=True)
df.drop('applause_rate', axis=1, inplace=True)
df = pd.get_dummies(df, columns=['tweets'], drop_first=True)



In [43]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Create linear regression object
regr_XGB = XGBRegressor(n_estimators=1000, learning_rate=0.05, random_state=1, max_depth=10, subsample=0.8, colsample_bytree=0.4, gamma = 0.1, reg_alpha = 0.1, reg_lambda = 0.1)

# Train the model using the training sets
X = df.drop('price', axis=1)
y = df['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12)
regr_XGB.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = regr_XGB.predict(X_test)

# Show RMSE, MAPE, R2
print('RMSE:', np.sqrt(mean_squared_error(y_test, y_pred)))
print('R2:', r2_score(y_test, y_pred))

RMSE: 295.82126177706385
R2: 0.3016119553270521


In [41]:
from sklearn.linear_model import LinearRegression
regr = LinearRegression()

regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)

# Show RMSE, MAPE, R2
print('RMSE:', np.sqrt(mean_squared_error(y_test, y_pred)))
print('R2:', r2_score(y_test, y_pred))

RMSE: 336.0005422917534
R2: 0.09901412461489567


In [44]:
import pickle

# save the trained model to a file using pickle
with open('model.pkl', 'wb') as f:
    pickle.dump(regr_XGB, f)